In [1]:
import pandas as pd
import stanza

ImportError: dlopen(/opt/anaconda3/envs/kg/lib/python3.8/site-packages/pyarrow/lib.cpython-38-darwin.so, 0x0002): Symbol not found: _deflate
  Referenced from: <DED14171-1B72-379D-A906-F88314F29201> /opt/anaconda3/envs/kg/lib/libarrow.800.0.0.dylib
  Expected in:     <47C6768E-CA90-32CD-87E6-865939F342B3> /opt/anaconda3/envs/kg/lib/liborc.dylib

In [ ]:


# Path to your dataset file
file_path = 'sample-1M.jsonl'

# Read only the first 100 lines to save memory
df = pd.read_json(file_path, lines=True, nrows=100)

print(f"Successfully loaded {len(df)} articles.")
if 'title' in df.columns:
    print("Sample Article Title:", df.loc[0, 'title'])
else:
    print("No 'title' column in the dataset")

# Display the first few rows
# print(df.head())

df.head()

In [ ]:

# # import pandas as pd

# # ---------------------------------------------------------
# # 1. Initialize Stanza Pipeline
# # ---------------------------------------------------------
# # 'processors': We only need tokenization and NER (Named Entity Recognition).
# # 'use_gpu': Set to True if you have a GPU, otherwise False.
# print("Initializing Stanza (downloading model if needed)...")
# stanza.download('en', processors='tokenize,ner')
# nlp = stanza.Pipeline('en', processors='tokenize,ner', use_gpu=False, verbose=False)

# # ---------------------------------------------------------
# # 2. Define Entity Extraction Function
# # ---------------------------------------------------------
# def get_entities_stanza(text):
#     """
#     Extracts specific entity types (PER, ORG, LOC, GPE) from text using Stanza.
#     """
#     print("1")
#     if not isinstance(text, str) or not text.strip():
#         return []
    
#     # Process text with Stanza
#     print("Process text with NLP")
#     doc = nlp(text)
    
#     # Filter for relevant entity types
#     # PERSON: People, including fictional.
#     # ORG: Companies, agencies, institutions.
#     # GPE: Countries, cities, states.
#     # LOC: Non-GPE locations, mountain ranges, bodies of water.
#     relevant_types = {'PERSON', 'ORG', 'GPE', 'LOC'}
    
#     entities = []
#     # Stanza processes text sentence by sentence
#     print("Stanza processes text sentence by sentence")
#     i = 1
#     for sent in doc.sentences:
#         # print("Sentence Number ", i)
#         # print("Sentence : ", sent)
#         for ent in sent.ents:
#             # print("Entities :", ent)
#             if ent.type in relevant_types:
#                 entities.append((ent.text, ent.type))
#                 # print("Type : ", ent.type)
#             # print("---------")
#         i = i + 1
#     print("Entities :", entities)
#     return entities

# # ---------------------------------------------------------
# # 3. Process the DataFrame
# # ---------------------------------------------------------
# print("Extracting entities from dataframe...")

# # Create a combined text column (Title + Content) for better context
# # Using .fillna('') to handle potential missing values
# print("If content in df.columns")
# if 'content' in df.columns:
#     df['full_text'] = df['title'].fillna('') + ". " + df['content'].fillna('')
# else:
#     df['full_text'] = df['title'].fillna('')

# # print(df['full_text'])

# # Apply the extraction function (This may take a minute for 100 rows)
# df['extracted_entities'] = df['full_text'].apply(get_entities_stanza)

# # ---------------------------------------------------------
# # 4. Display Results
# # ---------------------------------------------------------
# print("\n" + "="*60)
# print("STANZA NER EXTRACTION RESULTS")
# print("="*60)

# # Display the first 5 results
# for index, row in df.head(5).iterrows():
#     print(f"Article ID: {index}")
#     print(f"Title: {row.get('title', 'No Title')}")
#     print(f"Found Entities: {row['extracted_entities']}")
#     print("-" * 60)

In [ ]:
# ==========================================
# 2. LOAD GLiNER MODEL
# ==========================================
print("2. Loading GLiNER Model...")
# "urchade/gliner_medium-v2.1" is the best balance of speed vs accuracy
model = GLiNER.from_pretrained("urchade/gliner_medium-v2.1")

# ==========================================
# 3. DEFINE EXTRACTION FUNCTION
# ==========================================
def extract_entities(text):
    if not text: return []
    
    # DEFINE YOUR CUSTOM LABELS HERE
    # GLiNER allows you to invent any label you want!
    labels = ["Person", "Company", "City", "Country", "Product", "Crime"]
    
    entities = model.predict_entities(text, labels, threshold=0.3)
    
    # Return list of (Text, Label)
    return [(e['text'], e['label']) for e in entities]

# ==========================================
# 4. RUN EXTRACTION
# ==========================================
print("3. Extracting entities...")
# Combine title and text for better context
df['full_text'] = df['title'] + ". " + df['text']
df['entities'] = df['full_text'].apply(extract_entities)

# ==========================================
# 5. VIEW RESULTS
# ==========================================
for i, row in df.iterrows():
    print(f"\n--- Article {i} ---")
    print(f"TITLE: {row['title']}")
    print(f"ENTITIES FOUND: {row['entities']}")